In [35]:
"""Trains a Bayesian neural network to classify MNIST digits.

The architecture is LeNet-5 [1].

#### References

[1]: Yann LeCun, Leon Bottou, Yoshua Bengio, and Patrick Haffner.
     Gradient-based learning applied to document recognition.
     _Proceedings of the IEEE_, 1998.
     http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf
"""

import os
import warnings

# Dependency imports
from absl import app
from absl import flags
import matplotlib
matplotlib.use('Agg')
from matplotlib import figure  # pylint: disable=g-import-not-at-top
from matplotlib.backends import backend_agg
import numpy as np
# import tensorflow.compat.v2 as tf
import tensorflow as tf
import keras

# tf.enable_v2_behavior()

# TODO(b/78137893): Integration tests currently fail with seaborn imports.
warnings.simplefilter(action='ignore')

try:
  import seaborn as sns  # pylint: disable=g-import-not-at-top
  HAS_SEABORN = True
except ImportError:
  HAS_SEABORN = False

In [36]:


IMAGE_SHAPE = [28, 28, 1]
NUM_TRAIN_EXAMPLES = 60000
NUM_HELDOUT_EXAMPLES = 10000
NUM_CLASSES = 10

learning_rate = 0.001
num_epochs = 50
batch_size=128
data_dir=os.path.join(os.getenv('TEST_TMPDIR', '/tmp'), 'bayesian_neural_network/data')
model_dir = 'plots_mnist_mcdropout'
viz_steps=5
num_monte_carlo=50
fake_data = False

In [37]:
def plot_weight_posteriors(names, qm_vals, qs_vals, fname):
  """Save a PNG plot with histograms of weight means and stddevs.

  Args:
    names: A Python `iterable` of `str` variable names.
      qm_vals: A Python `iterable`, the same length as `names`,
      whose elements are Numpy `array`s, of any shape, containing
      posterior means of weight varibles.
    qs_vals: A Python `iterable`, the same length as `names`,
      whose elements are Numpy `array`s, of any shape, containing
      posterior standard deviations of weight varibles.
    fname: Python `str` filename to save the plot to.
  """
  fig = figure.Figure(figsize=(6, 3))
  canvas = backend_agg.FigureCanvasAgg(fig)

  ax = fig.add_subplot(1, 2, 1)
  for n, qm in zip(names, qm_vals):
    sns.distplot(qm.reshape([-1]), ax=ax, label=n)
  ax.set_title('weight means')
  ax.set_xlim([-1.5, 1.5])
  ax.legend()

  ax = fig.add_subplot(1, 2, 2)
  for n, qs in zip(names, qs_vals):
    sns.distplot(qs.reshape([-1]), ax=ax)
  ax.set_title('weight stddevs')
  ax.set_xlim([0, 1.])

  fig.tight_layout()
  canvas.print_figure(fname, format='png')
  print('saved {}'.format(fname))


def plot_heldout_prediction(input_vals, probs,
                            fname, n=10, title=''):
  """Save a PNG plot visualizing posterior uncertainty on heldout data.

  Args:
    input_vals: A `float`-like Numpy `array` of shape
      `[num_heldout] + IMAGE_SHAPE`, containing heldout input images.
    probs: A `float`-like Numpy array of shape `[num_monte_carlo,
      num_heldout, num_classes]` containing Monte Carlo samples of
      class probabilities for each heldout sample.
    fname: Python `str` filename to save the plot to.
    n: Python `int` number of datapoints to vizualize.
    title: Python `str` title for the plot.
  """
  fig = figure.Figure(figsize=(9, 3*n))
  canvas = backend_agg.FigureCanvasAgg(fig)
  for i in range(n):
    ax = fig.add_subplot(n, 3, 3*i + 1)
    ax.imshow(input_vals[i, :].reshape(IMAGE_SHAPE[:-1]), interpolation='None')

    ax = fig.add_subplot(n, 3, 3*i + 2)
    for prob_sample in probs:
      sns.barplot(np.arange(10), prob_sample[i, :], alpha=0.1, ax=ax)
      ax.set_ylim([0, 1])
    ax.set_title('posterior samples')

    ax = fig.add_subplot(n, 3, 3*i + 3)
    sns.barplot(np.arange(10), np.mean(probs[:, i, :], axis=0), ax=ax)
    ax.set_ylim([0, 1])
    ax.set_title('predictive probs')
  fig.suptitle(title)
  fig.tight_layout()

  canvas.print_figure(fname, format='png')
  print('saved {}'.format(fname))

In [38]:

class MNISTSequence(tf.keras.utils.Sequence):
  """Produces a sequence of MNIST digits with labels."""

  def __init__(self, data=None, batch_size=128, fake_data_size=None):
    """Initializes the sequence.

    Args:
      data: Tuple of numpy `array` instances, the first representing images and
            the second labels.
      batch_size: Integer, number of elements in each training batch.
      fake_data_size: Optional integer number of fake datapoints to generate.
    """
    if data:
      images, labels = data
    else:
      images, labels = MNISTSequence.__generate_fake_data(
          num_images=fake_data_size, num_classes=NUM_CLASSES)
    self.images, self.labels = MNISTSequence.__preprocessing(
        images, labels)
    self.batch_size = batch_size

  @staticmethod
  def __generate_fake_data(num_images, num_classes):
    """Generates fake data in the shape of the MNIST dataset for unittest.

    Args:
      num_images: Integer, the number of fake images to be generated.
      num_classes: Integer, the number of classes to be generate.
    Returns:
      images: Numpy `array` representing the fake image data. The
              shape of the array will be (num_images, 28, 28).
      labels: Numpy `array` of integers, where each entry will be
              assigned a unique integer.
    """
    images = np.random.randint(low=0, high=256,
                               size=(num_images, IMAGE_SHAPE[0],
                                     IMAGE_SHAPE[1]))
    labels = np.random.randint(low=0, high=num_classes,
                               size=num_images)
    return images, labels

  @staticmethod
  def __preprocessing(images, labels):
    """Preprocesses image and labels data.

    Args:
      images: Numpy `array` representing the image data.
      labels: Numpy `array` representing the labels data (range 0-9).

    Returns:
      images: Numpy `array` representing the image data, normalized
              and expanded for convolutional network input.
      labels: Numpy `array` representing the labels data (range 0-9),
              as one-hot (categorical) values.
    """
    images = 2 * (images / 255.) - 1.
    images = images[..., tf.newaxis]

    labels = tf.keras.utils.to_categorical(labels)
    return images, labels

  def __len__(self):
    return int(tf.math.ceil(len(self.images) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.images[idx * self.batch_size: (idx + 1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
    return batch_x, batch_y

In [39]:
  if tf.io.gfile.exists(model_dir):
    tf.compat.v1.logging.warning(
        'Warning: deleting old log directory at {}'.format(model_dir))
    tf.io.gfile.rmtree(model_dir)
  tf.io.gfile.makedirs(model_dir)

In [40]:
train_set, heldout_set = tf.keras.datasets.mnist.load_data()
import pandas as pd
labels = pd.Series(train_set[1])
labels.value_counts()

1    6742
7    6265
3    6131
2    5958
9    5949
0    5923
6    5918
8    5851
4    5842
5    5421
dtype: int64

In [41]:
where_out = np.where(train_set[1] == 3)
where_out
where_in = np.where(train_set[1] != 3)
where_in

(array([    0,     1,     2, ..., 59997, 59998, 59999], dtype=int64),)

In [42]:
train_out = train_set[0][where_out]
train_set_out = (train_set[0][where_out], train_set[1][where_out])
train_out.shape

(6131, 28, 28)

In [43]:
train_in = train_set[0][where_in]
train_set_in = (train_set[0][where_in], train_set[1][where_in])
train_in.shape

(53869, 28, 28)

In [44]:
train_set[0].shape

(60000, 28, 28)

In [45]:
heldout_y_srs = pd.Series(heldout_set[1])
heldout_y_srs.value_counts()

1    1135
2    1032
7    1028
3    1010
9    1009
4     982
0     980
8     974
6     958
5     892
dtype: int64

In [46]:
heldout_x = heldout_set[0]
heldout_y = heldout_set[1]
reorder_heldout = list()
for digit in range(10):
    where_digit = np.where(heldout_y == digit)
    reorder_heldout.append(where_digit[0][0])
reorder_heldout.extend(list(range(len(heldout_x))))
heldout_x = heldout_x[reorder_heldout]
heldout_y = heldout_y[reorder_heldout]
heldout_set = (heldout_x, heldout_y)

In [47]:
train_seq = MNISTSequence(data=train_set_in, batch_size=batch_size)
heldout_seq = MNISTSequence(data=heldout_set, batch_size=batch_size)

In [48]:
def create_model():
  """Creates a Keras model using the LeNet-5 architecture.

  Returns:
      model: Compiled Keras model.
  """
  # KL divergence weighted by the number of training samples, using
  # lambda function to pass as input to the kernel_divergence_fn on
  # flipout layers.

  # Define a LeNet-5 model using three convolutional (with max pooling)
  # and two fully connected dense layers. We use the Flipout
  # Monte Carlo estimator for these layers, which enables lower variance
  # stochastic gradients than naive reparameterization.

  model = tf.keras.models.Sequential([
      tf.keras.layers.Convolution2D(
          6, kernel_size=5, padding='SAME',
          activation=tf.nn.relu),
      tf.keras.layers.MaxPooling2D(
          pool_size=[2, 2], strides=[2, 2],
          padding='SAME'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Convolution2D(
          16, kernel_size=5, padding='SAME',
          activation=tf.nn.relu),
      tf.keras.layers.MaxPooling2D(
          pool_size=[2, 2], strides=[2, 2],
          padding='SAME'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Convolution2D(
          120, kernel_size=5, padding='SAME',
          activation=tf.nn.relu),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(
          84,
          activation=tf.nn.relu),
      tf.keras.layers.Dense(
          NUM_CLASSES,
          activation=tf.nn.softmax)
  ])


  optimizer = tf.keras.optimizers.Adam(lr= learning_rate)
  model.compile(optimizer, loss='categorical_crossentropy',
                metrics=['accuracy'], experimental_run_tf_function=False)
  return model

model = create_model()
model.build(input_shape=[None, 28, 28, 1])

In [49]:
num_epochs = 300
num_monte_carlo = 10
print(' ... Training convolutional neural network')
for epoch in range(num_epochs):
    print(epoch)
    epoch_accuracy, epoch_loss = [], []
    for step, (batch_x, batch_y) in enumerate(train_seq):
      batch_loss, batch_accuracy = model.train_on_batch(
          batch_x, batch_y)
      epoch_accuracy.append(batch_accuracy)
      epoch_loss.append(batch_loss)

      if step % 100 == 0:
        print('Epoch: {}, Batch index: {}, '
              'Loss: {:.3f}, Accuracy: {:.3f}'.format(
                  epoch, step,
                  tf.reduce_mean(epoch_loss),
                  tf.reduce_mean(epoch_accuracy)))

      if (step+1) % viz_steps == 0:
        # Compute log prob of heldout set by averaging draws from the model:
        # p(heldout | train) = int_model p(heldout|model) p(model|train)
        #                   ~= 1/n * sum_{i=1}^n p(heldout | model_i)
        # where model_i is a draw from the posterior p(model|train).
        print(' ... Running monte carlo inference')
        probs = tf.stack([model(heldout_x, training=True)
                          for _ in range(num_monte_carlo)], axis=0)
        mean_probs = tf.reduce_mean(probs, axis=0)
        heldout_log_prob = tf.reduce_mean(tf.math.log(mean_probs))
        print(' ... Held-out nats: {:.3f}'.format(heldout_log_prob))

        if HAS_SEABORN:
          names = [layer.name for layer in model.layers
                   if 'flipout' in layer.name]
          qm_vals = [layer.kernel_posterior.mean().numpy()
                     for layer in model.layers
                     if 'flipout' in layer.name]
          qs_vals = [layer.kernel_posterior.stddev().numpy()
                     for layer in model.layers
                     if 'flipout' in layer.name]
          plot_weight_posteriors(names, qm_vals, qs_vals,
                                 fname=os.path.join(
                                     model_dir,
                                     'epoch{}_step{:05d}_weights.png'.format(
                                         epoch, step)))
          plot_heldout_prediction(heldout_seq.images, probs.numpy(),
                                  fname=os.path.join(
                                      model_dir,
                                      'epoch{}_step{}_pred.png'.format(
                                          epoch, step)),
                                  title='mean heldout logprob {:.2f}'
                                  .format(heldout_log_prob))
model.save('mnist_nonbayes.h5')

 ... Training convolutional neural network
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266

In [50]:
model

In [52]:
num_monte_carlo = 5
probs = tf.stack([model(heldout_x, training=True)
                          for _ in range(num_monte_carlo)], axis=0)
mean_probs = tf.reduce_mean(probs, axis=0)
heldout_log_prob = tf.reduce_mean(tf.math.log(mean_probs))

model_dir = 'final_nonbayes'
plot_heldout_prediction(heldout_seq.images, probs.numpy(),
                                  fname=os.path.join(
                                      model_dir,
                                      'pred.png'.format(
                                          epoch, step)),
                                  title='mean heldout logprob {:.2f}'
                                  .format(heldout_log_prob))

saved final_nonbayes\pred.png


In [55]:
heldout_x = heldout_set[0]
heldout_y = heldout_set[1]
reorder_heldout = list()

where_digit = np.where(heldout_y == 3)[0].tolist()

heldout_x = heldout_x[where_digit]
heldout_y = heldout_y[where_digit]
heldout_set = (heldout_x, heldout_y)
heldout_seq = MNISTSequence(data = heldout_set, batch_size = batch_size)
where_digit = np.where(heldout_y == 3)[0].tolist()
num_monte_carlo = 100

probs = tf.stack([model(heldout_x, training=True)
                          for _ in range(num_monte_carlo)], axis=0)
mean_probs = tf.reduce_mean(probs, axis=0)
heldout_log_prob = tf.reduce_mean(tf.math.log(mean_probs))
for i in range(5):
    plot_heldout_prediction(heldout_seq.images[10 * i:10 * i + 10], probs.numpy()[:, 10 * i:10 * i + 10, :],
                            fname = os.path.join(
                                model_dir,
                                f'pred_{i}.png'.format(
                                    epoch, step)),
                            title = 'mean heldout logprob {:.2f}'
                            .format(heldout_log_prob))

saved final_nonbayes\pred_0.png
saved final_nonbayes\pred_1.png
saved final_nonbayes\pred_2.png
saved final_nonbayes\pred_3.png
saved final_nonbayes\pred_4.png
